In [1]:
import sys
sys.path.append('C:/Users/pradnya.gade/Projects')

from pydiSQL import pull_data
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Query to grab uk school term times
query = """
SELECT *
FROM analytics.uk_school_term_times;
"""

In [3]:
df = pull_data(query)

c:\ProgramData\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


Server connected via SSH


In [4]:
df.head(10)

,id,local_authority,term_info,date
0,1,Aberdeen City,term_1_end,2023-10-13
1,2,Aberdeen City,term_1_start,2023-08-22
2,3,Aberdeen City,term_2_end,2023-12-21
3,4,Aberdeen City,term_2_start,2023-10-30
4,5,Aberdeen City,term_3_end,2024-03-29
5,6,Aberdeen City,term_3_start,2024-01-08
6,7,Aberdeen City,term_4_end,2024-07-05
7,8,Aberdeen City,term_4_start,2024-04-15
8,9,Aberdeenshire,term_1_end,2023-10-13
9,10,Aberdeenshire,term_1_start,2023-08-22


In [7]:
df[df.term_info == 'term_1_start']

,id,local_authority,term_info,date
1,2,Aberdeen City,term_1_start,2023-08-22
9,10,Aberdeenshire,term_1_start,2023-08-22
17,18,Angus,term_1_start,2023-08-15
25,26,Argyll & Bute,term_1_start,2023-08-15
35,36,Barking & Dagenham,term_1_start,2023-09-04
...,...,...,...,...
2409,2410,Wokingham,term_1_start,2023-09-04
2421,2422,Wolverhampton,term_1_start,2023-09-05
2433,2434,Worcestershire,term_1_start,2023-09-04
2445,2446,Wrexham,term_1_start,2023-09-01


In [13]:
# Query to look at most popular counties in England
query = """
SELECT county_id,
       COUNT(county_id) AS num_of_users,
       tc.name
FROM analytics.dx_user AS du
	JOIN twinkl.twinkl_county AS tc
		ON du.county_id = tc.id
WHERE tc.country_id = 244
GROUP BY county_id
ORDER BY num_of_users DESC ;
"""

In [14]:
popular_counties = pull_data(query)

Server connected via SSH


In [15]:
popular_counties.head(10)

,county_id,num_of_users,name
0,54,160150,City of London
1,99,84541,West Midlands
2,35,68807,Essex
3,50,66759,Kent
4,101,56563,West Yorkshire
5,51,55274,Lancashire
6,93,53246,Surrey
7,45,50569,Hampshire
8,47,40793,Hertfordshire
9,11,40015,Cheshire


# Downloads
Looked for correlation between downloads and holiday time in the most five popular counties in England

In [84]:
# Grabbed all the England users who downloaded resources
query = """
SELECT du.user_id,
       du.country_id,
       du.county_id,
       gmtdl.datetime
FROM analytics.dx_user AS du
	JOIN analytics.george_meta_temp_download_log AS gmtdl 
		ON du.user_id = gmtdl.user_id
WHERE du.country_id = 244;
"""

In [85]:
england_download = pull_data(query)

Server connected via SSH


In [74]:
england_download

,user_id,country_id,county_id,date
0,168,244,3,2024-01-02
1,168,244,3,2024-01-02
2,192,244,11,2024-01-02
3,340,244,2,2024-01-02
4,340,244,2,2024-01-02
...,...,...,...,...
5913919,20517055,244,0,2024-01-15
5913920,20517074,244,0,2024-01-15
5913921,20517111,244,0,2024-01-15
5913922,20517252,244,0,2024-01-15


In [81]:
city_of_london = (
    england_download[england_download.county_id == 54]
    .groupby('date')['user_id']
    .count()
    .reset_index(name='number_of_downloads')
)

In [80]:
city_of_london

,date,number_of_downloads
0,2024-01-02,5162
1,2024-01-03,5772
2,2024-01-04,6094
3,2024-01-05,5781
4,2024-01-06,4420
5,2024-01-07,8755
6,2024-01-08,20000
7,2024-01-09,21201
8,2024-01-10,20494
9,2024-01-11,19596


In [66]:
df[df.local_authority == 'City of London']

,id,local_authority,term_info,date
412,413,City of London,term_1_end,2023-10-13
413,414,City of London,term_1_start,2023-09-04
414,415,City of London,term_2_end,2023-12-15
415,416,City of London,term_2_start,2023-10-30
416,417,City of London,term_3_end,2024-02-09
417,418,City of London,term_3_start,2024-01-08
418,419,City of London,term_4_end,2024-03-22
419,420,City of London,term_4_start,2024-02-19
420,421,City of London,term_5_end,2024-05-24
421,422,City of London,term_5_start,2024-04-15


In [75]:
england_download.date = pd.to_datetime(england_download.date)


In [67]:
import numpy as np

# Define the conditions
condition_holiday = ((city_of_london['date'] >= '2023-09-04') & (city_of_london['date'] <= '2023-10-13')) | ((city_of_london['date'] >= '2023-10-30') & (city_of_london['date'] <= '2023-12-15')) | ((city_of_london['date'] >= '2024-01-08') & (city_of_london['date'] <= '2024-02-09'))

# Create the 'holiday' column using numpy.where
city_of_london['Holiday'] = np.where(condition_holiday, 'N', 'Y')


In [68]:
city_of_london

,date,number_of_downloads,Holiday
0,2024-01-02,11345,Y
1,2024-01-03,13626,Y
2,2024-01-04,17117,Y
3,2024-01-05,16004,Y
4,2024-01-06,8970,Y
5,2024-01-07,14671,Y
6,2024-01-08,36320,N
7,2024-01-09,37930,N
8,2024-01-10,37868,N
9,2024-01-11,36081,N


# Apps data